<a href="https://colab.research.google.com/github/guide2157/ThaiLicensePlateRecognizer/blob/master/YoloV3Plate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Train YoloV3 for custom object

In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [0]:
%cd /content/drive/My Drive/darknet
!make
!chmod +x ./darknet

make: Nothing to be done for 'all'.


In [0]:
!sudo apt install dos2unix

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 31 not upgraded.
Need to get 351 kB of archives.
After this operation, 1,267 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 dos2unix amd64 7.3.4-3 [351 kB]
Fetched 351 kB in 1s (578 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package dos2unix.
(Reading database ... 144439 files and directories curren

In [0]:
!dos2unix ./data/train.txt
!dos2unix ./data/val.txt
!dos2unix ./data/yolo.data
!dos2unix ./data/yolo.names
!dos2unix ./cfg/yolov3_custom_train.cfg

dos2unix: converting file ./data/train.txt to Unix format...
dos2unix: converting file ./data/val.txt to Unix format...
dos2unix: converting file ./data/yolo.data to Unix format...
dos2unix: converting file ./data/yolo.names to Unix format...
dos2unix: converting file ./cfg/yolov3_custom_train.cfg to Unix format...


In [0]:
!./darknet detector train data/yolo.data cfg/yolov3_custom_train.cfg darknet53.conv.74

Streaming output truncated to the last 5000 lines.
Region 94 Avg IOU: 0.886344, Class: 0.999681, Obj: 0.997141, No Obj: 0.000239, .5R: 1.000000, .75R: 1.000000,  count: 1
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000001, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000003, .5R: -nan, .75R: -nan,  count: 0
Region 94 Avg IOU: 0.898478, Class: 0.999516, Obj: 0.996709, No Obj: 0.000351, .5R: 1.000000, .75R: 1.000000,  count: 2
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000001, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.835357, Class: 0.995216, Obj: 0.138143, No Obj: 0.000588, .5R: 1.000000, .75R: 1.000000,  count: 1
Region 94 Avg IOU: 0.810812, Class: 0.999864, Obj: 0.999652, No Obj: 0.000239, .5R: 1.000000, .75R: 1.000000,  count: 1
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000002, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.712926, Class: 0.996667, Obj: 0.544017, No Ob

Inference

In [0]:
weightsPath = '/content/drive/My Drive/darknet/backup/yolov3_custom_train_2000.weights'
configPath = '/content/drive/My Drive/darknet/cfg/yolov3_custom_train.cfg'

In [0]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

In [0]:
from google.colab.patches import cv2_imshow

In [0]:
net = cv2.dnn.readNetFromDarknet(configPath,weightsPath)

In [0]:
image = cv2.imread("/content/_MG_2341.jpg")
(H, W) = image.shape[:2]

In [0]:
LABELS = ['license']

In [0]:
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [0]:
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (608, 608), swapRB=True, crop=False)
net.setInput(blob)
layerOutputs = net.forward(ln)

In [0]:
threshold = 0.2

In [0]:
counter = 0

In [0]:
import glob 

images = glob.glob("/content/drive/My Drive/darknet/data/images/*.jpg")
images.extend(glob.glob('*.png'))
images.extend(glob.glob('*.JPG'))

In [0]:
for img in tqdm(images):
  predict(img)

In [0]:
def predict(img_path):
  img = cv2.imread(img_path)
  (H, W) = img.shape[:2]

  ln = net.getLayerNames()
  ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

  blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (608, 608), swapRB=True, crop=False)

  net.setInput(blob)
  layerOutputs = net.forward(ln)

  boxes = []
  confidences = []
  classIDs = []

  for output in layerOutputs:
	  for detection in output:
		  scores = detection[5:]
		  classID = np.argmax(scores)
		  confidence = scores[classID]
		  if confidence > threshold:
			  box = detection[0:4] * np.array([W, H, W, H])
			  (centerX, centerY, width, height) = box.astype("int")
			  x = int(centerX - (width / 2))
			  y = int(centerY - (height / 2))
			  boxes.append([x, y, int(width), int(height)])
			  confidences.append(float(confidence))
			  classIDs.append(classID)
     
  idxs = cv2.dnn.NMSBoxes(boxes, confidences, threshold, 0.1)
  global counter
  if len(idxs) > 0:
    cropped = firstCrop(img, boxes, idxs[0][0])
    chars_img = character_detection(cropped)
    for char in chars_img:
      cv2.imwrite("/content/drive/My Drive/CharsImg/{}.jpg".format(counter), char)
      counter+=1
  

In [0]:
def firstCrop(img, boxes, idx):
  (xtop, ytop) = (boxes[idx][0], boxes[idx][1])
  (w, h) = (boxes[idx][2], boxes[idx][3])
  firstCrop = img[ytop:ytop+h, xtop:xtop+w]
  return cv2.resize(firstCrop, (300, 100))

In [0]:
def auto_canny(image, sigma=0.33):
    v = np.median(image)

    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    return edged

In [0]:
def character_detection(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh_inv = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 39, 1)
    edges = auto_canny(thresh_inv)

    ctrs, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
    img_area = img.shape[0] * img.shape[1]
    charList = []

    for i, ctr in enumerate(sorted_ctrs):
        x, y, w, h = cv2.boundingRect(ctr)
        roi_area = w * h
        non_max_sup = roi_area / img_area

        if ((non_max_sup >= 0.015) and (non_max_sup < 0.09)):
            if ((h > w * 0.85) and (3 * w >= h)):
                char = img[y:y + h, x:x + w]
                resized = cv2.resize(char, (50,45))
                charList.append(char)
    return charList